# Northern Ireland UBI Feasibility Study

*Research supported by the UBI Lab Northern Ireland.*

* Amount: £200, 300, 400 per month
* Half shares for children and pensioners
* UBI included in neither means tests nor income tax
* Personal tax allowance at £2k
* Other rates from there, prioritizing higher 

## Tax and benefit policy in Northern Ireland


## Costing the UBI

* Cost of UBI up to £400 per month
* Start with amount from personal allowance
* Show equivalence between each rate increase


In [3]:
from policyengine.countries import UK
from openfisca_uk.model_api import *
from openfisca_uk.api import *
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import bisect

uk = UK()

_, baseline = uk._get_microsimulations(dict(baseline_country_specific="NORTHERN_IRELAND"))

def get_ubi_cost(
    adult_amount: float = 400 * 12,
    child_amount: float = 200 * 12,
    senior_amount: float = 100 * 12,
):
    in_ni = uk.baseline.calc("country", map_to="person") == "NORTHERN_IRELAND"
    num_adults = uk.baseline.calc("is_WA_adult")[in_ni].sum()
    num_children = uk.baseline.calc("is_child")[in_ni].sum()
    num_seniors = uk.baseline.calc("is_SP_age")[in_ni].sum()
    return adult_amount * num_adults + child_amount * num_children + senior_amount * num_seniors



In [19]:
def ubi_ni_reform(
    personal_allowance: float = 12_570,
    basic_rate_addition: float = 0,
    higher_rate_addition: float = 0,
    additional_rate_addition: float = 0,
) -> Reform:
    def modify_parameters(parameters):
        TIME_PERIOD = "year:2022:10"
        rates = parameters.tax.income_tax.rates.uk.brackets
        basic_rate = rates[0].rate
        higher_rate = rates[1].rate
        additional_rate = rates[2].rate
        INSTANT = "2022-01-01"
        basic_rate.update(period=TIME_PERIOD, value=basic_rate(INSTANT) + basic_rate_addition)
        higher_rate.update(period=TIME_PERIOD, value=higher_rate(INSTANT) + higher_rate_addition)
        additional_rate.update(period=TIME_PERIOD, value=additional_rate(INSTANT) + additional_rate_addition)
        pa = parameters.tax.income_tax.allowances.personal_allowance.amount
        pa.update(period=TIME_PERIOD, value=personal_allowance)
        return parameters

    class reform(Reform):
        def apply(self):
            self.modify_parameters(modify_parameters)
    
    return reform

def get_revenue(reform: Reform) -> float:
    baseline, reformed = uk._get_microsimulations(dict(reform=reform, baseline_country_specific="NORTHERN_IRELAND"))
    return baseline.calc("household_net_income").sum() - reformed.calc("household_net_income").sum()

def get_revenue_quick_estimate(
    personal_allowance: float,
    basic_rate_addition: float,
    higher_rate_addition: float,
    additional_rate_addition: float,
) -> float:
    income = baseline.calc("adjusted_net_income")
    weight = income.weights
    income_less_pa_b = np.maximum(0, income - 12_570)
    income_less_pa_r = np.maximum(0, income - personal_allowance)
    add_rate_income_b = np.maximum(0, income_less_pa_b - 150_000)
    add_rate_income_r = np.maximum(0, income_less_pa_r - 150_000)
    higher_rate_income_b = np.maximum(0, income_less_pa_b - 50_270) - add_rate_income_b
    higher_rate_income_r = np.maximum(0, income_less_pa_r - 50_270) - add_rate_income_r
    basic_rate_income_b = income_less_pa_b - higher_rate_income_b - add_rate_income_b
    basic_rate_income_r = income_less_pa_r - higher_rate_income_r - add_rate_income_r
    basic_rate_increased_tax = (
        basic_rate_income_r * (0.2 + basic_rate_addition)
        - basic_rate_income_b * 0.2
    )
    higher_rate_increased_tax = (
        higher_rate_income_r * (0.4 + higher_rate_addition)
        - higher_rate_income_b * 0.4
    )
    additional_rate_increased_tax = (
        add_rate_income_r * (0.45 + additional_rate_addition)
        - add_rate_income_b * 0.45
    )
    increased_tax = basic_rate_increased_tax + higher_rate_increased_tax + additional_rate_increased_tax
    return (increased_tax * weight).sum()

def solve_rates(
    adult_ubi: float = 0,
    personal_allowance: float = 12_570,
    higher_rate_ratio: float = 1,
    additional_rate_ratio: float = 1,
):
    def adjusted_reform_cost(
        basic_rate_addition: float = 0,
    ) -> float:
        """adjusted_reform = ubi_ni_reform(
            personal_allowance=personal_allowance,
            basic_rate_addition=basic_rate_addition,
            higher_rate_addition=higher_rate_ratio * basic_rate_addition,
            additional_rate_addition=additional_rate_ratio * basic_rate_addition,
        )"""
        adjusted_revenue = get_revenue_quick_estimate(personal_allowance, basic_rate_addition, basic_rate_addition * higher_rate_ratio, basic_rate_addition * additional_rate_ratio)
        ubi_cost = get_ubi_cost(adult_ubi, adult_ubi * 0.5, adult_ubi * 0.5)
        return ubi_cost - adjusted_revenue
    
    return bisect(adjusted_reform_cost, -1, 1, xtol=1e-3)

In [56]:
import numpy as np
from tqdm import tqdm

headline_ubi_rates = []
personal_allowances = []
basic_rate_additions = []
higher_add_addition_ratios = []

for ubi_rate in (200 * 12, 300 * 12, 400 * 12):
    for pa in (12_570, 10_000, 5_000, 2_000, 1_000, 0):
        for higher_add_ratio in (1, 1.25, 1.5):
            headline_ubi_rates.append(ubi_rate)
            personal_allowances.append(pa)
            basic_rate_additions.append(solve_rates(adult_ubi=ubi_rate, personal_allowance=pa, higher_rate_ratio=higher_add_ratio, additional_rate_ratio=higher_add_ratio))
            higher_add_addition_ratios.append(higher_add_ratio)

In [57]:
import pandas as pd

df = pd.DataFrame({
    "Adult UBI": headline_ubi_rates,
    "Personal Allowance": personal_allowances,
    "Basic rate addition": basic_rate_additions,
    "Higher/additional rate increase ratio": higher_add_addition_ratios,
})

df.to_csv("experiment_results.csv")

In [73]:
import plotly.express as px
from ubicenter import format_fig
from ubicenter.plotly import BLUE_COLOR_SEQUENCE

df["Personal Allowance reduction"] = 12_570 - df["Personal Allowance"]
df["Higher rate addition"] = df["Basic rate addition"] * df["Higher/additional rate increase ratio"]
df["Additional rate addition"] = df["Basic rate addition"] * df["Higher/additional rate increase ratio"]
df["UBI/Increase ratio combination"] = [
    f"£{ubi / 12:,.0f} per adult per month, higher/additional ratio {higher_add_ratio:.0%}"
    for ubi, higher_add_ratio in zip(
        df["Adult UBI"],
        df["Higher/additional rate increase ratio"],
    )
]
df["Label"] = [
    f"A £{ubi:,.0f} UBI and £{pa:,.0f} Personal Allowance can be made <br>budget neutral by setting the basic, higher and additional rates<br> to {0.2 + basic_rate_addition:.0%}, {0.4 + higher_rate_addition:.0%} and {0.45 + additional_rate_addition:.0%} respectively."
    for ubi, pa, basic_rate_addition, higher_rate_addition, additional_rate_addition in zip(
        df["Adult UBI"],
        df["Personal Allowance"],
        df["Basic rate addition"],
        df["Higher rate addition"],
        df["Additional rate addition"],
    )
]

df = df.sort_values(["Adult UBI", "Higher/additional rate increase ratio"])

fig = px.line(
    df, 
    x="Basic rate addition", 
    y="Personal Allowance reduction", 
    color="UBI/Increase ratio combination",  
    custom_data=[df.Label],
)
fig.update_layout(
    xaxis_tickformat=".0%",
    yaxis_tickprefix="£",
    yaxis_tickformat=",.0f",
    yaxis_range=(0, 12_570),
    xaxis_range=(0, 0.6),
    title="Budget neutral UBI funding models",
)

CUSTOM_HOVERTEMPLATE="%{customdata[0]}"
fig.update_traces(hovertemplate=CUSTOM_HOVERTEMPLATE)

for frame in fig.frames:
    for data in frame.data:
        data.hovertemplate = CUSTOM_HOVERTEMPLATE

format_fig(fig)

## Impacts of budget-neutral UBI plans

* Poverty
* Inequality
